<a href="https://colab.research.google.com/github/Raylynd6299/Intro-to-Learn-DRL/blob/main/Chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Policy Evaluation 

In [7]:
import numpy as np
import tensorflow as tf


In [4]:
P_BW = {
    0: {   #State- Example state 0
        0: [(1.0,0,0.0, True)], #Action (Action 0) : Possible transitions for that (State, Action)
        1: [(1.0,0,0.0, True)]  #[probabilit of that transition, next state, reward, flag thath indicated if the next state is terminal]
    },
    1: {
        0: [(1.0,0,0.0, True)],
        1: [(1.0,2,1.0, True)]
    },
    2: {
        0: [(1.0,2,0.0, True)],
        1: [(1.0,2,0.0, True)]
    }
}

In [5]:
P_FL = {
		0:	{
			0:	[ (0.6666666666666, 0, 0.0, False),
            (0.3333333333333, 4, 0.0, False),
          ],
			1:  [ (0.3333333333333, 0, 0.0, False),
            (0.3333333333333, 4, 0.0, False),
            (0.3333333333333, 1, 0.0, False),
          ],
			2:  [ (0.3333333333333, 0, 0.0, False),
            (0.3333333333333, 4, 0.0, False),
            (0.3333333333333, 1, 0.0, False),
          ],
			3: 	[ (0.6666666666666, 0, 0.0, False),
            (0.3333333333333, 1, 0.0, False),
          ],
			},
		1:	{
			0:	[ (0.3333333333333, 5, 0.0, False),
            (0.3333333333333, 0, 0.0, False),
            (0.3333333333333, 1, 0.0, False),
          ],
			1:  [ (0.3333333333333, 0, 0.0, False),
            (0.3333333333333, 2, 0.0, False),
            (0.3333333333333, 5, 0.0, False),
          ],
			2:  [ (0.3333333333333, 1, 0.0, False),
            (0.3333333333333, 2, 0.0, False),
            (0.3333333333333, 5, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 1, 0.0, False),
            (0.3333333333333, 0, 0.0, False),
            (0.3333333333333, 2, 0.0, False),
          ],
			},
		2:	{
			0:	[ (0.3333333333333, 2, 0.0, False),
            (0.3333333333333, 6, 0.0, False),
            (0.3333333333333, 1, 0.0, False),
          ],
			1:  [ (0.3333333333333, 1, 0.0, False),
            (0.3333333333333, 6, 0.0, False),
            (0.3333333333333, 3, 0.0, False),
          ],
			2:  [ (0.3333333333333, 3, 0.0, False),
            (0.3333333333333, 2, 0.0, False),
            (0.3333333333333, 6, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 1, 0.0, False),
            (0.3333333333333, 3, 0.0, False),
            (0.3333333333333, 2, 0.0, False),
          ],
			},
		3:	{
			0:	[ (0.3333333333333, 2, 0.0, False),
            (0.3333333333333, 3, 0.0, False),
            (0.3333333333333, 7, 0.0, False),
          ],
			1:  [ (0.3333333333333, 2, 0.0, False),
            (0.3333333333333, 7, 0.0, False),
            (0.3333333333333, 3, 0.0, False),
          ],
			2:  [ (0.3333333333333, 7, 0.0, False),
            (0.6666666666666, 3, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 2, 0.0, False),
            (0.6666666666666, 3, 0.0, False),
          ],
			},
		4: {
			0:	[ (0.3333333333333, 4, 0.0, False),
            (0.3333333333333, 0, 0.0, False),
            (0.3333333333333, 8, 0.0, False),
          ],
			1:  [ (0.3333333333333, 4, 0.0, False),
            (0.3333333333333, 8, 0.0, False),
            (0.3333333333333, 5, 0.0, False),
          ],
			2:  [ (0.3333333333333, 8, 0.0, False),
            (0.3333333333333, 5, 0.0, False),
            (0.3333333333333, 0, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 0, 0.0, False),
            (0.3333333333333, 4, 0.0, False),
            (0.3333333333333, 5, 0.0, False),
          ],
			},
		5: {
			0:	[ (1, 5, 0.0, False) ],
			1:	[ (1, 5, 0.0, False) ],
			2:	[ (1, 5, 0.0, False) ],
			3:	[ (1, 5, 0.0, False) ],
			},	
		6: {
			0:	[ (0.3333333333333, 2, 0.0, False),
					  (0.3333333333333, 5, 0.0, False),
					  (0.3333333333333, 10, 0.0, False),
				  ],
			1:  [ (0.3333333333333, 5, 0.0, False),
            (0.3333333333333, 7, 0.0, False),
            (0.3333333333333, 10, 0.0, False),
          ],
			2:  [ (0.3333333333333, 7, 0.0, False),
            (0.3333333333333, 2, 0.0, False),
            (0.3333333333333, 10, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 5, 0.0, False),
            (0.3333333333333, 2, 0.0, False),
            (0.3333333333333, 7, 0.0, False),
          ],
			},
		7: {
			0:	[ (1, 7, 0.0, False) ],
			1:	[ (1, 7, 0.0, False) ],
			2:	[ (1, 7, 0.0, False) ],
			3:	[ (1, 7, 0.0, False) ],
			},	
		8: {
			0:	[ (0.3333333333333, 8, 0.0, False),
            (0.3333333333333, 4, 0.0, False),
            (0.3333333333333, 12, 0.0, False),
          ],
			1:  [ (0.3333333333333, 8, 0.0, False),
            (0.3333333333333, 12, 0.0, False),
            (0.3333333333333, 9, 0.0, False),
          ],
			2:  [ (0.3333333333333, 9, 0.0, False),
            (0.3333333333333, 12, 0.0, False),
            (0.3333333333333, 4, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 8, 0.0, False),
            (0.3333333333333, 4, 0.0, False),
            (0.3333333333333, 9, 0.0, False),
          ],
			},
		9: {
			0:	[ (0.3333333333333, 8, 0.0, False),
            (0.3333333333333, 5, 0.0, False),
            (0.3333333333333, 13, 0.0, False),
          ],
			1:  [ (0.3333333333333, 8, 0.0, False),
            (0.3333333333333, 10, 0.0, False),
            (0.3333333333333, 13, 0.0, False),
          ],
			2:  [ (0.3333333333333, 5, 0.0, False),
            (0.3333333333333, 13, 0.0, False),
            (0.3333333333333, 10, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 8, 0.0, False),
            (0.3333333333333, 5, 0.0, False),
            (0.3333333333333, 10, 0.0, False),
          ],
			},
		10: {
			0:	[ (0.3333333333333, 9, 0.0, False),
            (0.3333333333333, 14, 0.0, False),
            (0.3333333333333, 6, 0.0, False),
          ],
			1:  [ (0.3333333333333, 9, 0.0, False),
            (0.3333333333333, 14, 0.0, False),
            (0.3333333333333, 11, 0.0, False),
          ],
			2:  [ (0.3333333333333, 14, 0.0, False),
            (0.3333333333333, 11, 0.0, False),
            (0.3333333333333, 6, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 6, 0.0, False),
            (0.3333333333333, 9, 0.0, False),
            (0.3333333333333, 11, 0.0, False),
          ],
			},
		11: {
			0:	[ (1, 11, 0.0, False) ],
			1:	[ (1, 11, 0.0, False) ],
			2:	[ (1, 11, 0.0, False) ],
			3:	[ (1, 11, 0.0, False) ],
			},
		12: {
			0:	[ (1, 12, 0.0, False) ],
			1:	[ (1, 12, 0.0, False) ],
			2:	[ (1, 12, 0.0, False) ],
			3:	[ (1, 12, 0.0, False) ],
			},
		13: {
			0:	[ (0.3333333333333, 9, 0.0, False),
            (0.3333333333333, 12, 0.0, False),
            (0.3333333333333, 13, 0.0, False),
          ],
			1:  [ (0.3333333333333, 12, 0.0, False),
            (0.3333333333333, 13, 0.0, False),
            (0.3333333333333, 14, 0.0, False),
          ],
			2:  [ (0.3333333333333, 14, 0.0, False),
            (0.3333333333333, 13, 0.0, False),
            (0.3333333333333, 9, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 9, 0.0, False),
            (0.3333333333333, 12, 0.0, False),
            (0.3333333333333, 14, 0.0, False),
          ],
			},
		14: {
			0:	[ (0.3333333333333, 13, 0.0, False),
            (0.3333333333333, 14, 0.0, False),
            (0.3333333333333, 10, 0.0, False),
          ],
			1:  [ (0.3333333333333, 15, 1.0, True ),
            (0.3333333333333, 13, 0.0, False),
            (0.3333333333333, 14, 0.0, False),
          ],
			2:  [ (0.3333333333333, 14, 0.0, False),
            (0.3333333333333, 15, 1.0, True ),
            (0.3333333333333, 10, 0.0, False),
          ],
			3: 	[ (0.3333333333333, 13, 0.0, False),
            (0.3333333333333, 10, 0.0, False),
            (0.3333333333333, 15, 1.0, True),
          ],
			},
		15: {
			0:	[ (1, 15, 0.0, True) ],
			1:	[ (1, 15, 0.0, True) ],
			2:	[ (1, 15, 0.0, True) ],
			3:	[ (1, 15, 0.0, True) ],
			}
	}


In [107]:
P_BSW = {
            0: {   #State- Example state 0
                0: [(1.0,0,0.0, True)], #Action (Action 0) : Possible transitions for that (State, Action)
                1: [(1.0,0,0.0, True)]  #[probabilit of that transition, next state, reward, flag thath indicated if the next state is terminal]
            },
            1: {
                0: [ ( 0.8,0,0.0, True ), ( 0.2,2,1.0, True ) ], #Stochastic
                1: [ ( 0.8,2,1.0, True ), ( 0.2,0,0.0, True ) ]
            },
            2: {
                0: [(1.0,2,0.0, True)],
                1: [(1.0,2,0.0, True)]
            }
        }

In [90]:
def printPolicyEval( ArrayFLMDP, policy ):
  if len(list(ArrayFLMDP)) != 16:
    return
  for index, value in enumerate(ArrayFLMDP):
    print(f"[{index},act = {policy(index)}] = {value}")

In [62]:
def policy_evaluation( pi, P, gamma=1.0, theta=1e-10 ):
  pre_V = np.zeros(len(P))
  numIter = 0
  while True:
    V = np.zeros(len(P))
    for s in range(len(P)):
      for prob, next_state, reward, done in P[s][pi(s)]:
        V[s] += prob * ( reward + gamma * pre_V[next_state] * (not done));
    if np.max(np.abs(pre_V - V)) < theta:
      break
    else:
      numIter += 1
    pre_V = V.copy()
  return pre_V, numIter

In [15]:
Direction = {"L": 0, "D": 1, "R": 2, "U":3}


In [91]:
pi_FL_Random = lambda s: {
    0: Direction["R"], 
    1: Direction["L"],
    2: Direction["D"],
    3: Direction["U"],
    4: Direction["L"],
    5: Direction["L"],
    6: Direction["R"],
    7: Direction["L"],
    8: Direction["U"],
    9: Direction["D"],
    10: Direction["U"],
    11: Direction["L"],
    12: Direction["L"],
    13: Direction["R"],
    14: Direction["D"],
    15: Direction["L"]
}[s]
Result_P_Eval_Random, numIter = policy_evaluation(pi=pi_FL_Random, P=P_FL, theta=1e-10)

printPolicyEval(Result_P_Eval_Random, pi_FL_Random)
print(f"Num Iteraciones = {numIter}")

[0,act = 2] = 0.11764705778090763
[1,act = 0] = 0.058823528823141055
[2,act = 1] = 0.05882352873740604
[3,act = 3] = 0.058823528537432716
[4,act = 0] = 0.1764705869771354
[5,act = 0] = 0.0
[6,act = 2] = 0.05882352900588036
[7,act = 0] = 0.0
[8,act = 3] = 0.2352941164611231
[9,act = 1] = 0.29411764621636016
[10,act = 3] = 0.11764705837306869
[11,act = 0] = 0.0
[12,act = 0] = 0.0
[13,act = 2] = 0.5294117640075989
[14,act = 1] = 0.7647058819586243
[15,act = 0] = 0.0
Num Iteraciones = 244


In [68]:
pi_FL_GoGetIt = lambda s: {
    0: Direction["R"], 
    1: Direction["R"],
    2: Direction["D"],
    3: Direction["L"],
    4: Direction["D"],
    5: Direction["L"],
    6: Direction["D"],
    7: Direction["L"],
    8: Direction["R"],
    9: Direction["R"],
    10: Direction["D"],
    11: Direction["L"],
    12: Direction["L"],
    13: Direction["R"],
    14: Direction["R"],
    15: Direction["L"]
}[s]
Result_P_Eval_GoGetIt, numIter = policy_evaluation( pi=pi_FL_GoGetIt, P=P_FL )

printPolicyEval(Result_P_Eval_GoGetIt, pi_FL_Random)
print(f"Num Iteraciones = {numIter}")

[0,act = 2] = 0.037499999788090856
[1,act = 0] = 0.024999999876243996
[2,act = 1] = 0.04999999985509304
[3,act = 3] = 0.024999999876243996
[4,act = 0] = 0.049999999875692785
[5,act = 0] = 0.0
[6,act = 2] = 0.09999999993294441
[7,act = 0] = 0.0
[8,act = 3] = 0.09999999985452872
[9,act = 1] = 0.24999999980859222
[10,act = 3] = 0.29999999985446874
[11,act = 0] = 0.0
[12,act = 0] = 0.0
[13,act = 2] = 0.44999999973011656
[14,act = 1] = 0.6499999998755728
[15,act = 0] = 0.0
Num Iteraciones = 67


In [69]:
pi_FL_Careful = lambda s: {
    0: Direction["L"], 
    1: Direction["U"],
    2: Direction["U"],
    3: Direction["U"],
    4: Direction["L"],
    5: Direction["L"],
    6: Direction["U"],
    7: Direction["L"],
    8: Direction["U"],
    9: Direction["D"],
    10: Direction["L"],
    11: Direction["L"],
    12: Direction["L"],
    13: Direction["R"],
    14: Direction["R"],
    15: Direction["L"]
}[s]
Result_P_Eval_Careful, numIter = policy_evaluation( pi=pi_FL_Careful, P=P_FL)

printPolicyEval(Result_P_Eval_Careful, pi_FL_Careful)
print(f"Num Iteraciones = {numIter}")

[0,act = 0] = 0.5999999977965138
[1,act = 3] = 0.599999996999603
[2,act = 3] = 0.5999999964319132
[3,act = 3] = 0.5999999961367988
[4,act = 0] = 0.5999999979648764
[5,act = 0] = 0.0
[6,act = 3] = 0.19999999877956037
[7,act = 0] = 0.0
[8,act = 3] = 0.599999998288749
[9,act = 1] = 0.5999999987434088
[10,act = 0] = 0.5199999989694374
[11,act = 0] = 0.0
[12,act = 0] = 0.0
[13,act = 2] = 0.679999999068124
[14,act = 2] = 0.7599999994641584
[15,act = 0] = 0.0
Num Iteraciones = 757


In [59]:
def policy_improvement(V, P, gamma=1.0):
  Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
  for s in range(len(P)):
    for a in range(len(P[s])):
      for prob, next_state, reward, done in P[s][a]:
        Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
  new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
  return new_pi

In [73]:
def printPiFunction( stateIndexArray, piFunction ):
  Direction = { 0:"L", 1:"D", 2:"R", 3:"U"}
  for stateIndex in stateIndexArray:
    print(f"[{stateIndex}] = {Direction[piFunction(stateIndex)]}")

In [76]:
## Improve Random Policy
V_function_Value_Random_policy = Result_P_Eval_Random
CarefulPlusPolicy1 = policy_improvement(V = V_function_Value_Random_policy, P = P_FL)

printPiFunction([index for index in range(len(P_FL))],CarefulPlusPolicy1)


[0] = L
[1] = U
[2] = L
[3] = U
[4] = L
[5] = L
[6] = L
[7] = L
[8] = U
[9] = D
[10] = L
[11] = L
[12] = L
[13] = R
[14] = D
[15] = L


In [77]:
## Improve GoGetIt Policy
V_function_Value_GoGetIt_policy = Result_P_Eval_GoGetIt
CarefulPlusPolicy2 = policy_improvement(V = V_function_Value_GoGetIt_policy, P = P_FL)

printPiFunction([index for index in range(len(P_FL))],CarefulPlusPolicy2)

[0] = L
[1] = U
[2] = R
[3] = U
[4] = L
[5] = L
[6] = L
[7] = L
[8] = U
[9] = D
[10] = L
[11] = L
[12] = L
[13] = R
[14] = D
[15] = L


In [78]:
## Improve Careful Policy
V_function_Value_Careful_policy = Result_P_Eval_Careful
CarefulPlusPolicy3 = policy_improvement(V = V_function_Value_Careful_policy, P = P_FL)

printPiFunction([index for index in range(len(P_FL))],CarefulPlusPolicy3)

[0] = L
[1] = U
[2] = U
[3] = U
[4] = L
[5] = L
[6] = L
[7] = L
[8] = U
[9] = D
[10] = L
[11] = L
[12] = L
[13] = R
[14] = D
[15] = L


In [102]:
def policy_iteration(P, gamma=1.0, theta=1e-10):
  random_actions = np.random.choice(tuple(P[0].keys()), len(P))
  ## Get Random policy
  pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]
  while True:
    old_pi = {s:pi(s) for s in range(len(P))}
    V,_ = policy_evaluation(pi, P, gamma, theta)
    pi = policy_improvement(V, P, gamma)
    if old_pi == {s:pi(s) for s in range(len(P))}:
      break
  return V, pi

In [105]:
## Get the best policy P PL
improveValue, improvePiPolicy = policy_iteration( P = P_FL )

printPolicyEval(Result_P_Eval_Careful, pi_FL_Careful)
printPiFunction([index for index in range(len(P_FL))],improvePiPolicy)

[0,act = 0] = 0.5999999977965138
[1,act = 3] = 0.599999996999603
[2,act = 3] = 0.5999999964319132
[3,act = 3] = 0.5999999961367988
[4,act = 0] = 0.5999999979648764
[5,act = 0] = 0.0
[6,act = 3] = 0.19999999877956037
[7,act = 0] = 0.0
[8,act = 3] = 0.599999998288749
[9,act = 1] = 0.5999999987434088
[10,act = 0] = 0.5199999989694374
[11,act = 0] = 0.0
[12,act = 0] = 0.0
[13,act = 2] = 0.679999999068124
[14,act = 2] = 0.7599999994641584
[15,act = 0] = 0.0
[0] = L
[1] = U
[2] = U
[3] = U
[4] = L
[5] = L
[6] = L
[7] = L
[8] = U
[9] = D
[10] = L
[11] = L
[12] = L
[13] = R
[14] = D
[15] = L


In [109]:
## Get the best policy P BSW
improveValue, improvePiPolicy = policy_iteration( P = P_BSW )

printPolicyEval(Result_P_Eval_Careful, pi_FL_Careful)
printPiFunction([index for index in range(len(P_BSW))],improvePiPolicy)

[0,act = 0] = 0.5999999977965138
[1,act = 3] = 0.599999996999603
[2,act = 3] = 0.5999999964319132
[3,act = 3] = 0.5999999961367988
[4,act = 0] = 0.5999999979648764
[5,act = 0] = 0.0
[6,act = 3] = 0.19999999877956037
[7,act = 0] = 0.0
[8,act = 3] = 0.599999998288749
[9,act = 1] = 0.5999999987434088
[10,act = 0] = 0.5199999989694374
[11,act = 0] = 0.0
[12,act = 0] = 0.0
[13,act = 2] = 0.679999999068124
[14,act = 2] = 0.7599999994641584
[15,act = 0] = 0.0
[0] = L
[1] = D
[2] = L


In [ ]:
def value_iteration(P, gamma=1.0, theta=1e-10):
  V = np.zeros(len(P), dtype=np.float64)
  while True:
    Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
    for s in range(len(P)):
      for a in range(len(P[s])):
        for prob, next_state, reward, done in P[s][a]:
          Q[s][a] += prob * (reward + gamma * V[next_state] * (not done))
    if np.max(np.abs(V - np.max(Q, axis=1))) < theta:
      break
    V = np.max(Q, axis=1)
  pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
  return V, pi